In [13]:
import glob
import json

In [20]:
filenames = glob.glob("./*.json")
file_dt = [file[10:-5] for file in filenames]
print(sorted(file_dt))
# for file in sorted(glob.glob("./*.json")):
#     with open(file, 'r') as f:
#         x = json.loads(f.read())
#         print(file, len(x))


['', '202441-1642', '202441-1643', '202441-1647', '202441-1653', '202441-1710', '202441-1727', '202441-1735', '202441-1743', '202441-1753', '202441-1810', '202441-1826', '202441-1834', '202441-1843', '202441-1853', '202441-1920', '202441-1925', '202441-1933', '202441-1943', '202441-1953', '202441-197', '202441-2010', '202441-2024', '202441-2043', '202441-2054', '202441-2120', '202441-2127', '202441-2135', '202441-2143', '202441-2156', '202441-218', '202441-2225', '202441-2233', '202441-2243', '202441-2253', '202441-229', '202441-2310', '202441-2323', '202441-2333', '202441-2343', '202441-2353', '202442-038', '202442-1011', '202442-1028', '202442-1036', '202442-1044', '202442-1053', '202442-1118', '202442-1125', '202442-1134', '202442-1143', '202442-1153', '202442-119', '202442-1215', '202442-1243', '202442-1258', '202442-1312', '202442-1325', '202442-1333', '202442-1343', '202442-1353', '202442-140', '202442-1424', '202442-1434', '202442-1444', '202442-1454', '202442-148', '202442-1523